In [1]:
import pandas as pd



In [2]:
train = pd.read_csv("train_tweets.csv")

print("Traininng Set:"% train.columns, train.shape, len(train))

test = pd.read_csv('test_tweets.csv')

print("Test Set:"% test.columns, test.shape, len(test))


Traininng Set: (31962, 3) 31962
Test Set: (17197, 2) 17197


### Text Cleaning

In [3]:
import re

# function for cleaning text 
def clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
    return df

test_clean = clean_text(test, 'tweet')
train_clean = clean_text(train, 'tweet')



In [7]:
train.head(5)

,id,label,tweet
0,1,0,when a father is dysfunctional and is so sel...
1,2,0,thanks for lyft credit i cant use cause they...
2,3,0,bihday your majesty
3,4,0,model i love u take with u all the time in u...
4,5,0,factsguide society now motivation


## Handling imbalanced classes

In [8]:
from sklearn.utils import resample

train_majority = train_clean[train_clean.label==0]
train_minority = train_clean[train_clean.label==1]

train_minority_upsampled = resample(train_minority,
                                   replace=True,
                                   n_samples=len(train_majority),
                                   random_state=1989)

train_upsampled = pd.concat([train_minority_upsampled, train_majority])
train_upsampled['label'].value_counts()

1    29720
0    29720
Name: label, dtype: int64

## Training

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier

pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('nb', SGDClassifier()),
])

##### Split training data into training and test set

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_upsampled['tweet'], train_upsampled['label'], random_state = 0)

####### fit model to training data and F1 Score

In [33]:
model = pipeline_sgd.fit(X_train, y_train)
y_predict = model.predict(X_test)
from sklearn.metrics import f1_score
f1_score(y_test, y_predict)


0.967285351849379